# Laphet with Myanmar Names Dataset

ဒီ Jupyter notebook က Laphet LM Toolkit ကို သုံးပြီး MLP, Bi-LSTM, Transformer, BERT, GPT အခြေခံတဲ့ language model (LM) တွေကို ဆောက်ကြည့်တာ၊ ဆောက်ထားပြီးသား LM ကိုသုံးပြီး text generation စမ်းလုပ်ပြထားတာပါ။  



## Dataset Information

Dataset link: [https://github.com/ye-kyaw-thu/myRoman](https://github.com/ye-kyaw-thu/myRoman)  

myRoman ဒေတာ format က အောက်ပါအတိုင်း ရှိတာမို့  

1	ကကြိုးကြာ	က ကြိုး ကြာ	ka kyoe kyar  
2	ကချောင်လွမ်းနန်း	က ချောင် လွမ်း နန်း	ka chaung lwan nan  
3	ကချောင်လွမ်းနန်း	က ချောင် လွမ်း နန်း	ka chaung lwan nang  

ဝဏ္ဏဖြတ်ထားတဲ့ (i.e. syllable) တတိယ ကော်လံကို ယူသုံးထားပါတယ်။  
ပြီးတော့ ‌ZWNJ (U+200C) နဲ့ SHY (U+00AD) သင်္ကေတတွေကို clean လုပ်ပြီး သုံးခဲ့ပါတယ်။  

In [1]:
ls ./data/myRoman/

dev_name.txt  start_names.txt  test_name.txt  train_name.txt


train_name.txt ဖိုင်က training လုပ်တဲ့အခါမှာ သုံးဖို့အတွက် ပြင်ထားတာ။  
dev_name.txt ဖိုင်က development သို့မဟုတ် validation အတွက် သုံးဖို့ ပြင်ထားတာ။  
test_name.txt ဖိုင်က testing/evaluation လုပ်ဖို့အတွက် ပြင်ထားတာ။  
start_names.txt ဖိုင်က ဆောက်ထားတဲ့ language model တွေနဲ့ မြန်မာနာမည်တွေကို generate လုပ်ခိုင်းတဲ့အခါမှာ ကိုယ်က စစေချင်တဲ့ ဝဏ္ဏ (i.e. syllable) စာလုံးတွေကို ပြင်ထားတဲ့ ဖိုင်ပါ။  

In [8]:
!wc ./data/myRoman/*.txt

   1000    3193   37204 ./data/myRoman/dev_name.txt
     15      18     205 ./data/myRoman/start_names.txt
   1000    3246   37728 ./data/myRoman/test_name.txt
  27246   88007 1028564 ./data/myRoman/train_name.txt
  29261   94464 1103701 total


အထက်မှာ မြင်ရတဲ့အတိုင်း training data က စာကြောင်းရေ နှစ်သောင်းခုနှစ်ထောင်ကျော်ရှိပါတယ်။  
development နဲ့ test ဒေတာကိုတော့ စာကြောင်းရေ တစ်ထောင်စီ ထားထားပါတယ်။

File format က sentence တစ်ကြောင်းကို နာမည်တစ်ခုမို့လို့ တနည်းအားဖြင့် training ဒေတာက မြန်မာစာနဲ့ ရိုက်ထားတဲ့ နာမည်ပေါင်း နှစ်သောင်းခုနှစ်ထောင်ကျော်ရှိပါတယ်။  

In [10]:
!head ./data/myRoman/train_name.txt

သူ ရ ဖြိုး မင်း
ခန့် မင်း သူ
ခိုင် ဇာ ဝင်း
ကြည် အောင် မင်း
အောင် ဇင် ခန့်
ကျော် လင်း ဝေ
ခတ္တာ ဇော်
ကောင်း ခန့် လင်း
အောင် ထူး
အောင် သန်း ထွန်း


In [11]:
!head ./data/myRoman/dev_name.txt

ကံ ကောင်း
လဲ့ လဲ့
ကျော် တိုး ဝေ
သက် ဘုန်း နိုင်
သိမ့် သူ သူ ကျော်
သဲ အိန္ဒြေ အောင်
ကျော် သန်း ထွန်း
အိ မွန် သန့်
အေး မြင့် မိုရ် ပိုင်
အောင် ဖုန်း ပိုင်


In [12]:
!head ./data/myRoman/test_name.txt

အေး သိင်္ဂီ
သာ စော
ကျော့ ယ မင်း သွယ်
မိုး ဆု ပန်
ခေါ် ထန့်
ခိုင် ခိုင် ဖြိုး
ကြယ် စင် မင်း ခန့်
သေး မယ်
သိဒ္ဓိ ဆန်း
က လျာ သန်း


In [13]:
!head ./data/myRoman/start_names.txt

ကျော်
မ မ
အေး
လှ လှ

မြ အေး
သ
မောင်
မြင့် မြင့်
ရွှေ


start_names.txt ဖိုင်ထဲမှာ blank line ကိုလည်း တမင်တကာ ထားပြီး စမ်းထားတာပါ။ ကျပန်း အစစာလုံးနဲ့ ပရိုဂရမ်က name generate လုပ်ပေးသလား စမ်းချင်လို့ပါ။  

## --help of Laphet

In [14]:
!python ./laphet.py --help

usage: laphet.py [-h] --model_type {mlp,bilstm,transformer,bert,gpt} [--train]
                 [--generate] [--test] [--data DATA] [--model MODEL]
                 [--vocab VOCAB] [--dev_file DEV_FILE] [--test_file TEST_FILE]
                 [--prompt PROMPT] [--input INPUT] [--seq_len SEQ_LEN]
                 [--output OUTPUT] [--no_of_generation NO_OF_GENERATION]
                 [--epochs EPOCHS] [--batch_size BATCH_SIZE] [--lr LR]
                 [--embed_dim EMBED_DIM] [--num_heads NUM_HEADS]
                 [--num_layers NUM_LAYERS] [--hidden_dim HIDDEN_DIM]
                 [--ff_dim FF_DIM] [--dropout DROPOUT]
                 [--temperature TEMPERATURE] [--top_k TOP_K] [--top_p TOP_P]

Laphet language model toolkit for Burmese.

options:
  -h, --help            show this help message and exit
  --model_type {mlp,bilstm,transformer,bert,gpt}
                        Type of model to use: mlp, bilstm, transformer, bert
                        or gpt.
  --train               

ကိုယ်စက်ထဲမှာ pytorch အပါအဝင် လိုအပ်တဲ့ python library တွေကို မှန်မှန်ကန်ကန် install လုပ်ထားရင် အထက်ပါလိုမျိုး help screen ကို မြင်ရပါလိမ့်မယ်။   

## Multilayer Perceptron (MLP) based Language Modeling

အရင်ဆုံး MLP-based LM ကို ဆောက်မယ်။ ပြီးရင် text generation လုပ်ကြည့်မယ်။ ပြီးရင် test data ကိုသုံးပြီး evaluation လုပ်ကြည့်မယ်။  

## Training

In [15]:
!time python laphet.py --model_type mlp --train --data ./data/myRoman/train_name.txt \
--dev_file ./data/myRoman/dev_name.txt --model ./model/name/mlp.model --seq_len 50 --epochs 10 --batch_size 32 --lr 0.0001

Epoch 1/10 (Training): 100%|█████████████████| 852/852 [00:06<00:00, 129.71it/s]
Epoch 1, Training Loss: 1.2911
Epoch 1/10 (Validation): 100%|█████████████████| 32/32 [00:00<00:00, 505.43it/s]
Epoch 1, Validation Loss: 1.0546
Best model saved at ./model/name/mlp.model with validation loss: 1.0546
Epoch 2/10 (Training): 100%|█████████████████| 852/852 [00:06<00:00, 133.31it/s]
Epoch 2, Training Loss: 1.0532
Epoch 2/10 (Validation): 100%|█████████████████| 32/32 [00:00<00:00, 504.22it/s]
Epoch 2, Validation Loss: 1.0525
Best model saved at ./model/name/mlp.model with validation loss: 1.0525
Epoch 3/10 (Training): 100%|█████████████████| 852/852 [00:06<00:00, 135.22it/s]
Epoch 3, Training Loss: 1.0522
Epoch 3/10 (Validation): 100%|█████████████████| 32/32 [00:00<00:00, 502.92it/s]
Epoch 3, Validation Loss: 1.0521
Best model saved at ./model/name/mlp.model with validation loss: 1.0521
Epoch 4/10 (Training): 100%|█████████████████| 852/852 [00:06<00:00, 137.78it/s]
Epoch 4, Training Loss: 1

ထွက်လာတဲ့ vocab ဖိုင်နဲ့ model ဖိုင်ကို လေ့လာကြည့်ရအောင်...

In [17]:
ls -lh ./model/name/mlp*

-rw-rw-r-- 1 ye ye 4.8M Jan 27 00:39 ./model/name/mlp.model
-rw-rw-r-- 1 ye ye  24K Jan 27 00:38 ./model/name/mlp.model.vocab


In [19]:
!file ./model/name/mlp.model

./model/name/mlp.model: Zip archive data, at least v0.0 to extract, compression method=store


In [21]:
!head ./model/name/mlp.model.vocab

ရု	0
တြီ	1
အဲ	2
မိန်	3
ဆုန်	4
တုံ	5
ကိုက်	6
စုက္က	7
ရှုံး	8
ကျော်	9


In [22]:
!tail ./model/name/mlp.model.vocab

ဝန်း	1433
ဘု	1434
အေ	1435
ကတ္တီ	1436
သိမ့်	1437
ဆံ	1438
မှိုင်	1439
အန်	1440
ရစ္စ	1441
ဇော	1442


## Text Generation

In [23]:
!time python -u laphet.py --model_type mlp --generate --model ./model/name/mlp.model \
--seq_len 50 --prompt "ရဲ" --no_of_generation 10

/home/ye/exp/name-lm/lib/laphet.py:114: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(args.model)
Generated Text 1: ရဲ သဉ္ဇာ ဘာ လှေ ဒွေး ချော ရို လျိ

လူနာမည်ဆိုရင်တော့ ဝဏ္ဏအလုံး ၅၀ ဆိုတာက မဖြစ်နိုင်လို့ နှစ်လုံးလောက်ပဲ ထားပြီး text generate လုပ်ခိုင်းကြည့်ရအောင်။   
sequence length ကို --seq_len 2 ဆိုတဲ့ setting နဲ့ run မယ်။ ရလဒ်က အောက်ပါအတိုင်းပါ။   
တစ်ခုရှိတာက တစ်ခေါက် generate လုပ်ခိုင်းတိုင်း နာမည်အသစ်တွေကိုပဲ generate လုပ်မှာမို့ run တဲ့အခေါက်တိုင်းမှာ မတူတဲ့ syllable sequence တွေကိုပဲ ထုတ်ပေးပါလိမ့်မယ်။  

In [25]:
!time python -u laphet.py --model_type mlp --generate --model ./model/name/mlp.model \
--seq_len 2 --prompt "ရဲ" --no_of_generation 10

/home/ye/exp/name-lm/lib/laphet.py:114: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(args.model)
Generated Text 1: ရဲ ဂျမ်း ရည်
Generated Text 2: ရဲ

myRoman ရဲ့ ဒေတာမှာက တိုင်းရင်းသားနာမည်မျိုးစုံ ပါဝင်ပါတယ်။ အဲဒါကြောင့် ဗမာနာမည်အနေနဲ့ ကြည့်ရင် ထူးဆန်းတဲ့ နာမည်မျိုးတွေလည်း ထုတ်ပေးပါလိမ့်မယ်။ သေချာတာက MLP-based language model က အလုပ်လုပ်ပေးတာကိုတော့ တွေ့ရပါလိမ့်မယ်။  

## Text Generation with Start-Word Input File

အင်္ဂလိပ်လိုခေါင်းစဉ်မှာက start word လို့ပေးထားပေမဲ့ ကျွန်တော်တို့ လက်ရှိ training လုပ်ထားတာက syllable သို့မဟုတ် ဝဏ္ဏဖြတ်ပြီး လုပ်ထားတာမို့လို့ start word သတ်မှတ်ပေးထားတဲ့ ဖိုင်မှာလည်း syllable unit ဖြတ်ပေးထားမှ အဆင်ပြေပါလိမ့်မယ်။ word ဖြတ်ပြီး training လုပ်ထားတဲ့ မော်ဒယ်အတွက်ဆိုရင်တော့ word unit နဲ့ သွားပါ။  

In [26]:
!time python laphet.py --model_type mlp --generate --model ./model/name/mlp.model \
--seq_len 2 --input ./data/myRoman/start_names.txt --no_of_generation 5 --output ./output/name/mlp_gen_texts.txt

/home/ye/exp/name-lm/lib/laphet.py:114: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(args.model)
Random Prompt Generated: သင်း
Generated texts saved

ထွက်လာတဲ့ output ဖိုင်ကို လေ့လာကြည့်ရအောင်။  
--no_of_generation 5 ဆိုပြီး ထားခဲ့တာမို့လို့ input file ထဲက အစ စာလုံး တစ်ခုစီအတွက် ဥပမာ ငါးခုစီ generate လုပ်ပေးပါလိမ့်မယ်။  

In [27]:
!cat ./output/name/mlp_gen_texts.txt

ကျော် သျှန္တီ နွမ်း
ကျော် ကာ ရို
ကျော် ဆပ် အေ
ကျော် ရုံး မြဉ္ဇူ
ကျော် ချိ လတ်
မ မ နု ထင်
မ မ ဖွာ ပျို
မ မ ဆူ အော
မ မ ဇို ဏာ
မ မ ခြူ ကြိုင်း
အေး ခံ့ အေး
အေး လိမ်း ရှိုင်းန်း
အေး ငိုင် လှီး
အေး စို ရေး
အေး အမ်း ဗန်
လှ လှ ဂျယ်လ် ကူ
လှ လှ ကွဲ ချိုင်း
လှ လှ ကြား စက္ကန့်
လှ လှ တီးလ် ဆွမ်း
လှ လှ ဇမ် ကူ
သင်း ကော စင်
သင်း ပေး အိပ်
သင်း ဇောင်း မုခ်
သင်း အက် ယံ
သင်း ကမ္မ လန်
မြ အေး တန့် ဒမ်း
မြ အေး လင် တေး
မြ အေး သက် ဒို့ဗ်
မြ အေး ဆယ် လှန်
မြ အေး လိုင် စိုက်
သ မိုက် ညွှန်း
သ ခဲ ဣန္ဒာ
သ မက် ခြယ်
သ က ပေါက်
သ တင်း သုဒ္ဓေါ
မောင် ချွတ် နွေး
မောင် တည် ရိုင်းန်
မောင် ပွန်း ဝူ
မောင် တွာ ထန်း
မောင် ပီး အောန်
မြင့် မြင့် ဆွင် တုတ်
မြင့် မြင့် ချင် သို့
မြင့် မြင့် စမ်း နား
မြင့် မြင့် တုတ် နွယ်
မြင့် မြင့် အက် [PAD]
ရွှေ တု ဂျမ်
ရွှေ ခံ့ ဟိန်း
ရွှေ မိန် ကွန့်
ရွှေ ကြယ် လွှမ်း
ရွှေ ဆည်း ခြာ
အဂ္ဂ တွေး ဣန္ဒ
အဂ္ဂ ရှဲမ်း ဇီ
အဂ္ဂ လျှား မွန်း
အဂ္ဂ တူး ဟေး
အဂ္ဂ ရွတ် အုတ်
ဥက္ကာ ရတ် မြင်
ဥက္ကာ သို ဖူ
ဥက္ကာ တောက် ဣန္တာ
ဥက္ကာ ကု ဘောမ်
ဥက္ကာ ခေါင်း လီ
သိင်္ဂီ ယ ဘင်
သိင်္ဂီ ရှိန် ရိုင်
သိင်္ဂီ ကပ် ဇန္ဒား
သိင်္ဂီ ပါ ကြွေ

input ဖိုင်က မေ့နေနိုင်တာမို့ ပြန်ရိုက်ပြရရင် အောက်ပါအတိုင်းပါ။  

In [28]:
!cat ./data/myRoman/start_names.txt

ကျော်
မ မ
အေး
လှ လှ

မြ အေး
သ
မောင်
မြင့် မြင့်
ရွှေ
အဂ္ဂ
ဥက္ကာ
သိင်္ဂီ
မေ
ခိုင်


## Testing and Evaluation

Testing လုပ်ချင်တာမို့လို့ --test ဆိုတဲ့ command option နဲ့ run ရပါလိမ့်မယ်။  
test ဖိုင်ရှိတဲ့ path ကို ညွှန်းပေးရပါလိမ့်မယ်။  
--test_file ./data/myRoman/test_name.txt  

In [29]:
!time python laphet.py --model_type mlp --test --model ./model/name/mlp.model \
--test_file ./data/myRoman/test_name.txt --seq_len 50 --batch_size 64

/home/ye/exp/name-lm/lib/laphet.py:216: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(args.model)
Testing: 100%|██████████████████████████████████| 1

Laphet LM Toolkit မှာ Perplexity ရော၊ Cross-Entropy နဲ့ရော evaluation လုပ်ကြည့်ထားပါတယ်။  

## Bi-LSTM based Language Modeling

ဒီတစ်ခါတော့ command တစ်ခုစီကို အသေးစိတ်မရှင်းတော့ဘူး။  
--model_type ဆိုတဲ့ နေရာမှာ bilstm ကို setup လုပ်ရင်ရပါပြီ။  
ထွက်လာမယ့် model ဖိုင်နာမည်ကိုတော့ မတူအောင်ပေးသင့်တာပေါ့။  

## Training

In [31]:
!time python laphet.py --model_type bilstm --train --data ./data/myRoman/train_name.txt \
--dev_file ./data/myRoman/dev_name.txt --model ./model/name/bilstm.model --seq_len 50 --epochs 10 --batch_size 32 --lr 0.0001

Epoch 1/10 (Training): 100%|██████████████████| 852/852 [00:15<00:00, 55.18it/s]
Epoch 1, Training Loss: 0.4833
Epoch 1/10 (Validation): 100%|█████████████████| 32/32 [00:00<00:00, 164.94it/s]
Epoch 1, Validation Loss: 0.3169
Best model saved at ./model/name/bilstm.model with validation loss: 0.3169
Epoch 2/10 (Training): 100%|██████████████████| 852/852 [00:15<00:00, 56.16it/s]
Epoch 2, Training Loss: 0.3127
Epoch 2/10 (Validation): 100%|█████████████████| 32/32 [00:00<00:00, 164.77it/s]
Epoch 2, Validation Loss: 0.3085
Best model saved at ./model/name/bilstm.model with validation loss: 0.3085
Epoch 3/10 (Training): 100%|██████████████████| 852/852 [00:15<00:00, 56.40it/s]
Epoch 3, Training Loss: 0.3053
Epoch 3/10 (Validation): 100%|█████████████████| 32/32 [00:00<00:00, 164.33it/s]
Epoch 3, Validation Loss: 0.3010
Best model saved at ./model/name/bilstm.model with validation loss: 0.3010
Epoch 4/10 (Training): 100%|██████████████████| 852/852 [00:15<00:00, 56.37it/s]
Epoch 4, Trainin

In [32]:
!ls -lh ./model/name/bilstm.model*

-rw-rw-r-- 1 ye ye 92M Jan 27 01:11 ./model/name/bilstm.model
-rw-rw-r-- 1 ye ye 24K Jan 27 01:08 ./model/name/bilstm.model.vocab


## Text Generation

--prompt "ရဲ" ဆိုတဲ့ setting နဲ့ run မှာမို့လို့ "ရဲ" နဲ့ စတဲ့ နာမည်တွေကို generate လုပ်ပေးသွားပါလိမ့်မယ်။  
sequence length က ၅၀ ထားထားတာမို့လို့ နာမည်အရှည်ကြီးတွေ ရိုက်ထုတ်ပေးပါလိမ့်မယ်။  
ပြီးတော့ --no_of_generation ကို ၁၀ ထားထားတာမို့လို့ စုစုပေါင်း စာကြောင်း ၁၀ကြောင်းအနေနဲ့ generate လုပ်ပေးသွားပါလိမ့်မယ်။  

In [33]:
!time python laphet.py --model_type bilstm --generate --model ./model/name/bilstm.model \
--seq_len 50 --prompt "ရဲ" --no_of_generation 10

/home/ye/exp/name-lm/lib/laphet.py:114: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(args.model)
Generated Text 1: ရဲ ကွမ်း ကိန်း မြန် မြန် ထား ထား 

## Text Generation with Start-Word Input File

In [34]:
!time python laphet.py --model_type bilstm --generate --model ./model/name/bilstm.model \
--seq_len 2 --input ./data/myRoman/start_names.txt --no_of_generation 5 --output ./output/name/bilstm_gen_texts.txt

/home/ye/exp/name-lm/lib/laphet.py:114: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(args.model)
Random Prompt Generated: ကြား
Generated texts saved

In [35]:
!cat ./output/name/bilstm_gen_texts.txt

ကျော် ကိန်း မြန်
ကျော် မြန် မြန်
ကျော် ထား မြန်
ကျော် ကိန်း ထား
ကျော် မြန် မြန်
မ မ ချက်စ် ထား
မ မ မြန် ထား
မ မ သဉ္ဇ ချက်စ်
မ မ ချက်စ် ထား
မ မ ကိန်း ကိန်း
အေး မြန် ကိန်း
အေး မြန် မြန်
အေး မြန် မြန်
အေး ကိန်း မြန်
အေး မြန် ကိန်း
လှ လှ ဇ မြန်
လှ လှ ဇ မြန်
လှ လှ ချက်စ် ကွမ်း
လှ လှ ကိန်း သဉ္ဇ
လှ လှ ဆိုက် ချက်စ်
ကြား ဆိုက် မြန်
ကြား ထား ဇ
ကြား ဆိုက် သဉ္ဇ
ကြား ဇ ထား
ကြား ချက်စ် ချက်စ်
မြ အေး မြန် မြန်
မြ အေး မြန် ထား
မြ အေး မြန် မြန်
မြ အေး မြန် မြန်
မြ အေး မြန် ဇ
သ မြန် ချက်စ်
သ ဇ မြန်
သ သဉ္ဇ ထား
သ မြန် ချက်စ်
သ ချက်စ် ဇ
မောင် ချက်စ် ဇ
မောင် ကွမ်း ကွမ်း
မောင် ကိန်း မြန်
မောင် ချက်စ် သဉ္ဇ
မောင် ကိန်း ဇ
မြင့် မြင့် ဆိုက် သဉ္ဇ
မြင့် မြင့် ထား ကွမ်း
မြင့် မြင့် ဇ ဇ
မြင့် မြင့် ထား ကိန်း
မြင့် မြင့် ကွမ်း ကိန်း
ရွှေ မြန် မြန်
ရွှေ သဉ္ဇ ထား
ရွှေ သဉ္ဇ ဇ
ရွှေ ထား ဆိုက်
ရွှေ ဆိုက် ဇ
အဂ္ဂ ကိန်း ကွမ်း
အဂ္ဂ သဉ္ဇ ချက်စ်
အဂ္ဂ ကိန်း ချက်စ်
အဂ္ဂ ဇ ဇ
အဂ္ဂ ထား မြန်
ဥက္ကာ မြန် ကိန်း
ဥက္ကာ ကိန်း ဇ
ဥက္ကာ ဆိုက် ထား
ဥက္ကာ ဇ ကိန်း
ဥက္ကာ ကွမ်း ဆိုက်
သိင်္ဂီ ဇ ကိန်း
သိင်္ဂီ ချက်စ် မြန်
သိင်္ဂီ သဉ္ဇ မြန်
သိင်္ဂီ ဆိုက

In [36]:
!wc ./output/name/bilstm_gen_texts.txt

  75  245 3026 ./output/name/bilstm_gen_texts.txt


## Testing/Evaluation

In [37]:
!time python laphet.py --model_type bilstm --test --model ./model/name/bilstm.model \
--test_file ./data/myRoman/test_name.txt --seq_len 50 --batch_size 64

/home/ye/exp/name-lm/lib/laphet.py:216: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(args.model)
Testing: 100%|██████████████████████████████████| 1

# Transformer based Language Modeling 

## Training

In [38]:
!time python laphet.py --model_type transformer --train --data ./data/myRoman/train_name.txt \
--dev_file ./data/myRoman/dev_name.txt --model ./model/name/transformer.model --seq_len 50 --epochs 10 --batch_size 32 --lr 0.0001

Epoch 1/10 (Training): 100%|█████████████████| 852/852 [00:04<00:00, 194.39it/s]
Epoch 1, Training Loss: 0.2378
Epoch 1/10 (Validation): 100%|█████████████████| 32/32 [00:00<00:00, 589.61it/s]
Epoch 1, Validation Loss: 0.0437
Best model saved at ./model/name/transformer.model with validation loss: 0.0437
Epoch 2/10 (Training): 100%|█████████████████| 852/852 [00:04<00:00, 208.26it/s]
Epoch 2, Training Loss: 0.0290
Epoch 2/10 (Validation): 100%|█████████████████| 32/32 [00:00<00:00, 592.74it/s]
Epoch 2, Validation Loss: 0.0211
Best model saved at ./model/name/transformer.model with validation loss: 0.0211
Epoch 3/10 (Training): 100%|█████████████████| 852/852 [00:04<00:00, 209.82it/s]
Epoch 3, Training Loss: 0.0151
Epoch 3/10 (Validation): 100%|█████████████████| 32/32 [00:00<00:00, 589.92it/s]
Epoch 3, Validation Loss: 0.0135
Best model saved at ./model/name/transformer.model with validation loss: 0.0135
Epoch 4/10 (Training): 100%|█████████████████| 852/852 [00:04<00:00, 204.99it/s]
E

မော်ဒယ် အမျိုးအစားပေါ်ကို မူတည်ပြီးတော့ training လုပ်တဲ့အခါမှာ ကြာချိန်က တူမှာ မဟုတ်ပါဘူး။ လက်ရှိမှာ နာမည်ဒေတာမို့လို့ တကယ်က စာကြောင်းအနေနဲ့ တိုပါတယ်။ ပြီးတော့ သုံးထားတဲ့ GPU အမျိုးအစားနဲ့ memory အပေါ်မှာလည်း မူတည်ပါလိမ့်မယ်။  

In [39]:
!ls -lh ./model/name/transformer.model*

-rw-rw-r-- 1 ye ye 11M Jan 27 01:26 ./model/name/transformer.model
-rw-rw-r-- 1 ye ye 24K Jan 27 01:25 ./model/name/transformer.model.vocab


## Text Generation

In [40]:
!time python laphet.py --model_type transformer --generate --model ./model/name/transformer.model \
--seq_len 50 --prompt "ရဲ" --no_of_generation 10

/home/ye/exp/name-lm/lib/laphet.py:114: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(args.model)
Generated Text 1: ရဲ ဋာ သိင်္ဂီ စေး ငိုက် ညို ခဲ ဖေ

Transformer based LM ရဲ့ text generation ဥပမာတချို့ကတော့ အထက်မှာ မြင်ရတဲ့ အတိုင်းပါပဲ။  

## Text Generation with Start-Word Input File

In [41]:
!time python laphet.py --model_type transformer --generate --model ./model/name/transformer.model \
--seq_len 2 --input ./data/myRoman/start_names.txt --no_of_generation 5 --output ./output/name/transformer_gen_texts.txt

/home/ye/exp/name-lm/lib/laphet.py:114: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(args.model)
Random Prompt Generated: ခွါ
Generated texts saved 

--output option သုံးပြီးတော့ ထွက်လာတဲ့ generated sentence တွေကို ဖိုင်ထဲမှာ သိမ်းခိုင်းထားတာမို့လို့ အဲဒီဖိုင်ကို cat command နဲ့ ရိုက်ထုတ်ပြခိုင်းပြီး ဘယ်လို စာကြောင်းတွေ ထွက်လာသလဲ ဆိုတာကို လေ့လာကြည့်ကြရအောင်။  

In [42]:
!cat ./output/name/transformer_gen_texts.txt

ကျော် အဏ္ဏ ခွာ
ကျော် ရူ ထင်
ကျော် ခေါမ် ဆုံး
ကျော် စဉ့် ဖေ
ကျော် လစ် လောင်း
မ မ ဝိန် တက္ခ
မ မ ငဲ ဖေ
မ မ ဖာ နောင့်
မ မ လိန်း အွန်
မ မ ဟွေ ကွန်း
အေး နံ့ ဘောမ်
အေး လိမ်း ဖေ
အေး သန် အ
အေး ထွတ် ရေ့စ်
အေး သား ယား
လှ လှ သေ့ တက္ခ
လှ လှ ရဲ ဟမ်
လှ လှ နှစ် ချင်း
လှ လှ ရာဇ် ရွန်း
လှ လှ နောင့် သျှင်
ခွါ ဖေ အောင်
ခွါ ဖေ မောက်
ခွါ ဖေ သောင်း
ခွါ ဖေ ဝင်
ခွါ ဖေ လာန်
မြ အေး ယာ နဲမ်
မြ အေး ဦး မင်
မြ အေး လွှမ်း ဧ
မြ အေး ဆု ဘွိုင်
မြ အေး ဿာ ဖေ
သ ဋာ ဘု
သ ဖျား တွေး
သ ကျီး ဖေ
သ ဣန္တာ မော်
သ သင်္ကြန် သိုက်
မောင် ကျူ မိုင်းလ်
မောင် လမ့် ကစ်
မောင် ဒန့် ဖေ
မောင် အံ ဖေ
မောင် ခေါင် တောင်း
မြင့် မြင့် တက္ခ ဖေ
မြင့် မြင့် ကျိမ်း တက္ခ
မြင့် မြင့် အုတ် ဖေ
မြင့် မြင့် ခေါ်လ် မော်
မြင့် မြင့် လက္ခ ဖေ
ရွှေ စန္ဒီ ကွေး
ရွှေ ရှာ ဆွမ့်
ရွှေ ယိန်း မွိုင်
ရွှေ ဈာန် ဝါ
ရွှေ စီ ညွန်
အဂ္ဂ နဲ ဖေ
အဂ္ဂ သျှ ဖေ
အဂ္ဂ မြေး ဖေ
အဂ္ဂ ဖိုး ခွေး
အဂ္ဂ ဆမ်း ဖေ
ဥက္ကာ ကမ် ထန်း
ဥက္ကာ ဖီး ဟတ်
ဥက္ကာ ဂျိမ်းစ် ဖေ
ဥက္ကာ ရိုး တီး
ဥက္ကာ ကြည့် ဖေ
သိင်္ဂီ တုံ ရှဲမ်း
သိင်္ဂီ ဘိုင် ဘွမ်
သိင်္ဂီ လွေ ဖေ
သိင်္ဂီ အိန္ဒု ဂျို
သိင်္ဂီ ယန်း ရုဏ်း
မေ ကျန်း အက္ခ
မေ သေ

## Testing/Evaluation

In [43]:
!time python laphet.py --model_type transformer --test --model ./model/name/transformer.model \
--test_file ./data/myRoman/test_name.txt --seq_len 50 --batch_size 64

/home/ye/exp/name-lm/lib/laphet.py:216: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(args.model)
Testing: 100%|██████████████████████████████████| 1

## BERT based Language Modeling

## Training

In [44]:
!time python laphet.py --model_type bert --train --data ./data/myRoman/train_name.txt \
--dev_file ./data/myRoman/dev_name.txt --model ./model/name/bert.model --seq_len 50 --epochs 10 --batch_size 32 --lr 0.0001

Epoch 1/10 (Training): 100%|█████████████████| 852/852 [00:04<00:00, 192.73it/s]
Epoch 1, Training Loss: 0.2482
Epoch 1/10 (Validation): 100%|█████████████████| 32/32 [00:00<00:00, 588.18it/s]
Epoch 1, Validation Loss: 0.0448
Best model saved at ./model/name/bert.model with validation loss: 0.0448
Epoch 2/10 (Training): 100%|█████████████████| 852/852 [00:04<00:00, 210.17it/s]
Epoch 2, Training Loss: 0.0292
Epoch 2/10 (Validation): 100%|█████████████████| 32/32 [00:00<00:00, 608.11it/s]
Epoch 2, Validation Loss: 0.0217
Best model saved at ./model/name/bert.model with validation loss: 0.0217
Epoch 3/10 (Training): 100%|█████████████████| 852/852 [00:04<00:00, 210.53it/s]
Epoch 3, Training Loss: 0.0151
Epoch 3/10 (Validation): 100%|█████████████████| 32/32 [00:00<00:00, 597.23it/s]
Epoch 3, Validation Loss: 0.0139
Best model saved at ./model/name/bert.model with validation loss: 0.0139
Epoch 4/10 (Training): 100%|█████████████████| 852/852 [00:04<00:00, 210.04it/s]
Epoch 4, Training Loss

In [45]:
!ls -lh ./model/name/bert*

-rw-rw-r-- 1 ye ye 11M Jan 27 01:39 ./model/name/bert.model
-rw-rw-r-- 1 ye ye 24K Jan 27 01:38 ./model/name/bert.model.vocab


## Text Generation

In [47]:
!time python laphet.py --model_type bert --generate --model ./model/name/bert.model \
--seq_len 50 --prompt "ရဲ" --no_of_generation 10

/home/ye/exp/name-lm/lib/laphet.py:114: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(args.model)
Generated Text 1: ရဲ ဘွယ် အဲင်း ချူး ခြူး ဖူ နန်းဒ်

ဒီတခါတော့ "ကျော်" နဲ့ စတဲ့ နာမည်တွေကို generate လုပ်ခိုင်းကြည့်ရအောင်။ sequence length ကိုတော့ 5 ထားထားတယ်။  

In [49]:
!time python laphet.py --model_type bert --generate --model ./model/name/bert.model \
--seq_len 5 --prompt "ကျော်" --no_of_generation 10

/home/ye/exp/name-lm/lib/laphet.py:114: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(args.model)
Generated Text 1: ကျော် တိုက် ဿ ဒါ ရဲ့ ကွယ်
Generat

ဒီတခါတော့ "မြင့်" နဲ့ စတဲ့ နာမည်တွေကို generate လုပ်ခိုင်းကြည့်ရအောင်။ sequence length ကိုတော့ 2 ပေးထားတယ်။  

In [51]:
!time python laphet.py --model_type bert --generate --model ./model/name/bert.model \
--seq_len 2 --prompt "မြင့်" --no_of_generation 10

/home/ye/exp/name-lm/lib/laphet.py:114: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(args.model)
Generated Text 1: မြင့် ဇဉ် ရေ့စ်
Generated Text 2:

## Text Generation with Start-Word Input File

In [52]:
!time python laphet.py --model_type bert --generate --model ./model/name/bert.model \
--seq_len 2 --input ./data/myRoman/start_names.txt --no_of_generation 5 --output ./output/name/bert_gen_texts.txt

/home/ye/exp/name-lm/lib/laphet.py:114: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(args.model)
Random Prompt Generated: ခတ္တာ
Generated texts save

In [53]:
!cat ./output/name/bert_gen_texts.txt

ကျော် တင့် ပေါ
ကျော် ဘက် ညှင်း
ကျော် ဖေါ ချူး
ကျော် သက် အဉ္ဇူ
ကျော် ဣန္ဒ ချူး
မ မ မ ကြိုင်း
မ မ တုပ် ချူး
မ မ ဖယ် ချူး
မ မ မင်း လာဘ်
မ မ ကျိန်း ချူး
အေး မတ် ယော်
အေး ဇာရ် ချူး
အေး စက္ကန့် ချူး
အေး သု ယက်
အေး မြော် နှဲမ်း
လှ လှ တက္ခ ချူး
လှ လှ တင့် လောင်ဝ်
လှ လှ ကောင်း ဒေါသ်
လှ လှ ဖယ် ချူး
လှ လှ သီ မိုင်
ခတ္တာ အဉ္ဇူ ရဲ
ခတ္တာ ဿန် ချူ
ခတ္တာ ဆော် ဝါ
ခတ္တာ ခိ ကြိုင်
ခတ္တာ သင်္ကြန် ထောင်
မြ အေး ရှမ်း အိမ်
မြ အေး ကတ် ဂိမ်း
မြ အေး တေး ချူး
မြ အေး ဒါ စင်း
မြ အေး လွန် လော
သ ဆားရ် ဆို
သ ရှိ ကြည့်
သ ဖေါ ချူး
သ သျှန်း ရန်း
သ ကန်း ချူး
မောင် ဂန္တ ဟွမ်
မောင် ခတ္တာ စဲ
မောင် ရိုး ဖိုး
မောင် ခင် လှဲန်
မောင် သက် ယား
မြင့် မြင့် ရေ့စ် ချူး
မြင့် မြင့် ပေါ့ ညှာ
မြင့် မြင့် မီးရ် ဿ
မြင့် မြင့် ကိုင်း ကောလ်
မြင့် မြင့် ဝဏ္ဏ ကျွံ
ရွှေ ပါ ကံ့
ရွှေ ရှမ် ချူး
ရွှေ အဉ္ဇ စို
ရွှေ ကျီ ပိ
ရွှေ ဝင့် လို့
အဂ္ဂ ကွန့် ကပ်
အဂ္ဂ မြော် နှဲမ်း
အဂ္ဂ ကျောက် ကိန္န
အဂ္ဂ နွန် ကယ်လ်
အဂ္ဂ ကောင်း လှန်
ဥက္ကာ ဠု ချူး
ဥက္ကာ ဂျ ပွဲ
ဥက္ကာ ဖု နှဲမ်း
ဥက္ကာ အက္ခ ချူး
ဥက္ကာ လွမ် ရှိုး
သိင်္ဂီ တန့် အဲလ်
သိင်္ဂီ ဇိန်း ချူး
သိင်္ဂီ လာဘ် မိုက်


## Testing/Evaluation

In [54]:
!time python laphet.py --model_type bert --test --model ./model/name/bert.model \
--test_file ./data/myRoman/test_name.txt --seq_len 50 --batch_size 64

/home/ye/exp/name-lm/lib/laphet.py:216: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(args.model)
Testing: 100%|██████████████████████████████████| 1

## GPT based Language Modeling

## Training

In [55]:
!time python laphet.py --model_type gpt --train --data ./data/myRoman/train_name.txt \
--dev_file ./data/myRoman/dev_name.txt --model ./model/name/gpt.model --seq_len 50 --epochs 10 --batch_size 32 --lr 0.0001

Epoch 1/10 (Training): 100%|█████████████████| 852/852 [00:04<00:00, 200.41it/s]
Epoch 1, Training Loss: 0.0759
Epoch 1/10 (Validation): 100%|█████████████████| 32/32 [00:00<00:00, 579.82it/s]
Epoch 1, Validation Loss: 0.0018
Best model saved at ./model/name/gpt.model with validation loss: 0.0018
Epoch 2/10 (Training): 100%|█████████████████| 852/852 [00:03<00:00, 213.35it/s]
Epoch 2, Training Loss: 0.0012
Epoch 2/10 (Validation): 100%|█████████████████| 32/32 [00:00<00:00, 590.19it/s]
Epoch 2, Validation Loss: 0.0005
Best model saved at ./model/name/gpt.model with validation loss: 0.0005
Epoch 3/10 (Training): 100%|█████████████████| 852/852 [00:04<00:00, 204.90it/s]
Epoch 3, Training Loss: 0.0004
Epoch 3/10 (Validation): 100%|█████████████████| 32/32 [00:00<00:00, 561.17it/s]
Epoch 3, Validation Loss: 0.0002
Best model saved at ./model/name/gpt.model with validation loss: 0.0002
Epoch 4/10 (Training): 100%|█████████████████| 852/852 [00:04<00:00, 209.78it/s]
Epoch 4, Training Loss: 0

In [56]:
!ls -lh ./model/name/gpt*

-rw-rw-r-- 1 ye ye 11M Jan 27 01:51 ./model/name/gpt.model
-rw-rw-r-- 1 ye ye 24K Jan 27 01:51 ./model/name/gpt.model.vocab


## Text Generation

In [57]:
!time python laphet.py --model_type gpt --generate --model ./model/name/gpt.model \
--seq_len 50 --prompt "ရဲ" --no_of_generation 10

/home/ye/exp/name-lm/lib/laphet.py:114: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(args.model)
Generated Text 1: ရဲ ပူး မစ် ကျူ ချန် မော် ကို မြ ဖ

ဒီတခါတော့ --prompt အေး ဆိုတာနဲ့ --seq_len 2 ပဲ ထားပြီး generate လုပ်ခိုင်းကြည့်ရအောင်။  

In [60]:
!time python laphet.py --model_type gpt --generate --model ./model/name/gpt.model \
--seq_len 2 --prompt "အေး" --no_of_generation 10

/home/ye/exp/name-lm/lib/laphet.py:114: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(args.model)
Generated Text 1: အေး ဆဲလ် ရှိုင်းန်း
Generated Tex

ဒီတစ်ခါတော့ "မြင့်" နဲ့ စတဲ့ နာမည်တွေကို generate လုပ်ခိုင်းကြည့်မယ်။  
sequence length ကို 2 ထားမယ်။ ပြီးတော့ no of generation ကိုတော့ 5 ပဲ ထားမယ်။  

In [61]:
!time python laphet.py --model_type gpt --generate --model ./model/name/gpt.model \
--seq_len 2 --prompt "မြင့်" --no_of_generation 5

/home/ye/exp/name-lm/lib/laphet.py:114: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(args.model)
Generated Text 1: မြင့် ကြိုး ကိုလ်
Generated Text 

## Text Generation with Start-Word Input File

In [62]:
!time python laphet.py --model_type gpt --generate --model ./model/name/gpt.model \
--seq_len 2 --input ./data/myRoman/start_names.txt --no_of_generation 5 --output ./output/name/gpt_gen_texts.txt

/home/ye/exp/name-lm/lib/laphet.py:114: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(args.model)
Random Prompt Generated: သော်
Generated texts saved

Save လုပ်ထားတဲ့ gpt_gen_texts.txt ဖိုင်ကို ကြည့်ကြည့်ရအောင်။  

In [63]:
!cat ./output/name/gpt_gen_texts.txt

ကျော် တည် စက္ကန့်
ကျော် ဗြာ ဥက္ကာ
ကျော် ကန် ကွက်
ကျော် ညိန်း ရယ်လ်
ကျော် ဝမ် ငြိမ်
မ မ ခါ ကျေး
မ မ သု လဲန်း
မ မ တုတ် လစ်
မ မ ကျက် ဘီ
မ မ နွမ် ငွေ့
အေး ထား ခေါင်း
အေး စင် ဂုံ
အေး လာ စုက္က
အေး အိန္ဒြာ ချားလ်
အေး နဲမ် လွှမ်း
လှ လှ လျှန် ဒူ
လှ လှ ဟင်း ပါယ်
လှ လှ ထဲမ်း အိမ်း
လှ လှ ခေါ်လ် ရိုင်း
လှ လှ ခန် စိမ့်
သော် ဘန်း ဇေ
သော် ပြာ ဒ
သော် ပြောင်း ညွှန်း
သော် ဗုံ ဇမ်
သော် နာ့ ညိမ့်
မြ အေး တွမ် ဝယ်လ်
မြ အေး ထောင်း ထန်း
မြ အေး ကွာ မြင်
မြ အေး လျှမ် ထန်
မြ အေး ကိမ်း ဇင်
သ ဆန့် ဂျူး
သ ရှိုင်းန်း ဘွယ်
သ တိန့် ကျော့
သ ကဉ္စ ခံ့
သ လွန်း ကျဲ
မောင် ဗာ လှဲ
မောင် ကင်းမ် ကူး
မောင် ကန်း ပါယ်
မောင် အောင်း သို့
မောင် ဗြာ ကွပ်
မြင့် မြင့် နွမ် ရင်
မြင့် မြင့် ရော် ဏ
မြင့် မြင့် ကောန် ဇီ
မြင့် မြင့် နဲမ်း ပုံ
မြင့် မြင့် စုံ ပံ
ရွှေ ဇူ ရှိန်
ရွှေ နှင်း ဂဂ်
ရွှေ ပွန်း ချောက်
ရွှေ စိုး ညို့
ရွှေ လင်္ကာ ရင်း
အဂ္ဂ လိုင်း အန့်
အဂ္ဂ မိုင် တွန်း
အဂ္ဂ အဉ္ဇ ကာ
အဂ္ဂ လွှင် ကျိမ်း
အဂ္ဂ ခြာ ဇမ့်
ဥက္ကာ ပယ်လ် ဂျန်
ဥက္ကာ အဲင်း ဇာရ်
ဥက္ကာ လဲန်း သျွှန်း
ဥက္ကာ ဝင်း ယှဉ်
ဥက္ကာ ကျော လ
သိင်္ဂီ နှိုင်း ရေး
သိင်္ဂီ သစ္စာ ဖေ
သိင်္ဂီ 

## Bash Shell Script

just for your knowledge ပါ။  
အထက်မှာ ပြခဲ့တဲ့အတိုင်း မော်ဒယ် တစ်ခုစီကို coding လုပ်ရင်းနဲ့ စမ်းရတဲ့အခါမှာ အကြိမ်ကြိမ်အခါခါ run လိုက် coding ကို ဝင်ပြင်လိုက်လုပ်ရတာမို့လို့ command line argument တွေအတိုင်းအတာ တစ်ခုထိ သတ်မှတ်ပြီးသွားတဲ့အခါမှာ shell script ရေးလိုက်ပြီး run ပြီးမှ log တွေ၊ ထွက်လာတဲ့ output တွေကို manual ဝင်စစ်ကြည့်တာမျိုး လုပ်ခဲ့ရပါတယ်။ အဲဒီလို လုပ်မှလည်း command တွေကို ရိုက်ထည့်နေရတဲ့ အချိန်သက်သာပါတယ်။  

လက်တွေ့ experiment တွေလုပ်တဲ့အခါမှာလည်း shel script ရေးတတ်ဖို့ လိုအပ်ပါတယ်။  

Laphet LM Toolkit ကို coding လုပ်နေစဉ်မှာ သုံးခဲ့တဲ့ bash shell script က အောက်ပါအတိုင်းပါ။  

In [64]:
!cat ./train_test_name.sh

#!/bin/bash

# Create the output and log directories if they don't exist
mkdir -p model/name/
mkdir -p output/name/
mkdir -p log/name/

# Function to train, generate text, and test a language model
task() {
  local model_type=$1
  local model_file="./model/name/${model_type}.model"
  local output_file="./output/name/${model_type}_gen_texts.txt"
  local log_file="./log/name/${model_type}.log"
  local train_data="./data/myRoman/train_name.txt"
  local dev_data="./data/myRoman/dev_name.txt"
  local test_data="./data/myRoman/test_name.txt"
  local start_name="./data/myRoman/start_names.txt"

  {
    echo "Training ${model_type^} language model:";
    time python -u laphet.py --model_type $model_type --train --data $train_data \
      --dev_file $dev_data --model $model_file --seq_len 50 --epochs 10 --batch_size 32 --lr 0.0001;

    echo "Text generation:";
    time python -u laphet.py --model_type $model_type --generate --model $model_file \
      --seq_len 50 --prompt "ရဲ" --no_of_generat

## Running MLP, Bi-LSTM, Transformer, BERT and GPT Language Models Building

ဒီတခါတော့ Myanmar name dataset ကိုသုံးပြီး မော်ဒယ် ၅မျိုးစလုံးရဲ့ training, text generation နဲ့ testing/evaluation အကုန် အစအဆုံးကို အထက်က shell script ကိုသုံးပြီး run ကြည့်ပါမယ်။   

In [65]:
!./train_test_name.sh

Training Mlp language model:
Epoch 1/10 (Training): 100%|█████████████████| 852/852 [00:06<00:00, 130.69it/s]
Epoch 1, Training Loss: 1.3057
Epoch 1/10 (Validation): 100%|█████████████████| 32/32 [00:00<00:00, 498.28it/s]
Epoch 1, Validation Loss: 1.0546
Best model saved at ./model/name/mlp.model with validation loss: 1.0546
Epoch 2/10 (Training): 100%|█████████████████| 852/852 [00:06<00:00, 138.64it/s]
Epoch 2, Training Loss: 1.0531
Epoch 2/10 (Validation): 100%|█████████████████| 32/32 [00:00<00:00, 511.96it/s]
Epoch 2, Validation Loss: 1.0525
Best model saved at ./model/name/mlp.model with validation loss: 1.0525
Epoch 3/10 (Training): 100%|█████████████████| 852/852 [00:06<00:00, 138.08it/s]
Epoch 3, Training Loss: 1.0522
Epoch 3/10 (Validation): 100%|█████████████████| 32/32 [00:00<00:00, 505.81it/s]
Epoch 3, Validation Loss: 1.0521
Best model saved at ./model/name/mlp.model with validation loss: 1.0521
Epoch 4/10 (Training): 100%|█████████████████| 852/852 [00:06<00:00, 136.86it

## GPU Usage Information

လက်ရှိ ရေးထားတဲ့ Laphet LM Toolkit က Pytorch library ကို သုံးထားပါတယ်။ Neural network ကို အခြေခံတဲ့ language modeling လုပ်တာမို့လို့ GPU ပေါ်မှာ run မှ မြန်ပါလိမ့်မယ်။ တကယ်လို့ GPU မရှိရင် ဒါမှမဟုတ် GPU က စက်မှာရှိနေပေမဲ့ setup လုပ်ထားတဲ့ Python environment က အကြောင်းတခုခုကြောင့် GPU ကို ခေါ်မသုံးနိုင်ရင်တော့ CPU ပေါ်မှာပဲ Laphet က run ပေးသွားပါလိမ့်မယ်။  

အထက်က Shell script ကို run နေစဉ်မှာ GPU usage ကို တချက်ကြည့်ကြည့်တော့ အောက်ပါအတိုင်းပါ။ 

## FYI

ဒီ notebook ကို အစကနေ အဆုံးထိ သေချာဖတ်လာတယ် ဆိုရင်တော့ NLP, AI အနည်းဆုံးတော့ မြန်မာစာကို သုံးပြီး language model ဆောက်တာနဲ့ ပတ်သက်ပြီး စိတ်ဝင်စားတယ်လို့ ယူဆပါတယ်။ 

တခု သိစေချင်တာက Laphet LM Toolkit က NLP/AI fundamental education အတွက် ရည်ရွယ်ပြီး တပတ်လောက်နဲ့ အကြမ်းပြီးအောင် ရေးထားတာမို့လို့ production အတွက် မသုံးစေချင်ပါဘူး။ Language modeling နဲ့ ပတ်သက်ပြီး ပထမဆုံး လေ့လာတဲ့သူတွေအတွက် လွယ်ကူအောင် တတ်နိုင်သမျှ ရှင်းရှင်းလေးရေးထားတာမို့ပါ။ အဲဒါတောင်မှ တကယ်တမ်း ရေးထားတဲ့ Laphet LM code အကုန် နဲ့ background theory ကို ထဲထဲဝင်ဝင် နားလည်ဖို့ ဆိုရင် ကျောင်းသားပေါ်မူတည်ပြီး သုံးလ၊ လေးလ ကြာသွားနိုင်ပါတယ်။ အနည်းဆုံးတော့ မြန်မာကျောင်းသားတွေအတွက်က မြန်မာနာမည်တွေကို သုံးပြီးတော့ မော်ဒယ်ဆောက်ပြ၊ စာကြောင်း အသစ်တွေကို generate လုပ်ပြ၊ testing/evaluation လုပ်ပြထားတာမို့လို့ ဒီ example notebook ကို အခြေခံပြီး NLP/AI သုတေသန အတွက် အရေးကြီးတဲ့ language modeling ဆိုတာကို နည်းနည်းပါးပါး တီးမိခေါက်မိရှိသွားရင်ပဲ ဝမ်းသာပါတယ်လို့။  

ရဲကျော်သူ  
LU Lab., Myanmar  
27 Jan 2025  
Email: ykt.nlp.ai@gmail.com